In [52]:

url = "https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N"
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

Members = soup.findAll(class_ = 'Members--bio u-richtext')
state_district = soup.findAll(class_ = 'Hero-title')
Money = soup.findAll(class_ = 'Members--number')


#### Get candidate and affiliation info 

In [53]:
def get_candidate_info(Members):
    members_list = []
    member_affiliation = []

    for i in range(len(Members)):
        if i == 0:
            name_affiliation = Members[0].text
            members_list.append(
                re.split(r"[\t\n()]", name_affiliation)[2].strip()
            )
            member_affiliation.append(
                re.split(r"[\t\n()]", name_affiliation)[3].strip()
            )
        else:
            
            name_affiliation = Members[i].text
            members_list.append(
                re.split(r"[\t()]", name_affiliation)[10].strip()
                )
            member_affiliation.append(
                re.split(r"[\t()]", name_affiliation)[11].strip()
                )
    return members_list, member_affiliation


### Return 1 for incumbent = True and 0 if incumbent = False

In [54]:
# def get_incumbent(Members):    
#     list = []
#     for th in Members:
#      #print(th.text.strip())
#         list.append(re.split(r"[\t\n()]", th.text))
#     incumbent_list = []
#     for i in range(len(list)):
#         if any("Incumbent" in item for item in list[i]):
#             incumbent_list.append(1)
#         else:
#             incumbent_list.append(0)
#     return incumbent_list

In [55]:
def get_incumbent(Members):    
    incumbent_list = []
    incumbent_list.append(["Incumbent" in Members[th].text for th in range(0,len(Members))])
    return incumbent_list[0]

#### Get percentage of votes obtained

for th in Members:
    print(th.text)

for th in Members:
     #print(th.text.strip())
     if '%' in th.text: 
          print(float(re.split(r"[()]", th.text)[3].split('%')[0]))
     else:
          print ("NAN")


In [56]:
def get_percentage(Members):
    percentage_list = []
    for th in Members:
        if '%' in th.text:
            percentage_list.append(float(re.split(r"[()]", th.text)[3].split('%')[0]))
        else:
            percentage_list.append(np.nan)
    return percentage_list

In [57]:
# def get_winner(Members):
#     percent_num = []
#     for th in Members:
#         percent_num.append(re.split(r"[(%)]", th.text)[3])
#     winner_list = []
#     for i in range(len(percent_num)):
#         if percent_num[i] == max(percent_num):
#             winner_list.append(1)
#         else:
#             winner_list.append(0)
#     return winner_list

def get_winner(Members):
    winner_list = []
    winner_list.append(["Winner" in Members[th].text for th in range(0,len(Members))])
        
    return winner_list[0]


In [58]:
def get_state_district(state_district):
    state_list = []
    district_list = []
    for i in range(len(Members)):
        state_list.append(state_district[0].text.split()[0])
        district_list.append(state_district[0].text.split()[2])

    return state_list, district_list

In [59]:
def get_money_details(Money):
    Money_list = []
    for i in range(len(Money)):
        Money_list.append(float(re.sub(",","", Money[i].text.split("$")[1])))
    Money_raised = Money_list[0::3]
    Money_spent = Money_list[1::3]
    return Money_raised, Money_spent

In [60]:
incumbent_list = get_incumbent(Members)
percentage_list = get_percentage(Members)
winner_list = get_winner(Members)
candidate_list = get_candidate_info(Members)
state_list = get_state_district(state_district)[0]
district_list = get_state_district(state_district)[1]
Money_details = get_money_details(Money)

In [61]:
percentage_list

[69.9, 27.3, 2.2, 0.6]

In [62]:
incumbent_list

[True, False, False, False]

In [63]:
candidate_df = pd.DataFrame({"Candidate Name" : candidate_list[0],
                             "Affiliation" : candidate_list[1],
                             "Incumbent": incumbent_list,
                             "Percentage of Vote" : percentage_list,
                             "Winner" : winner_list,
                             "State": state_list,
                             "District" : district_list,
                             "Money Raised": Money_details[0],
                             "Money Spent": Money_details[1]}
                             )

In [64]:
candidate_df

,Candidate Name,Affiliation,Incumbent,Percentage of Vote,Winner,State,District,Money Raised,Money Spent
0,Mark Green,R,True,69.9,True,Tennessee,07,1194960.0,935487.0
1,Kiran Sreepada,D,False,27.3,False,Tennessee,07,206644.0,207191.0
2,Ronald Brown,I,False,2.2,False,Tennessee,07,1750.0,0.0
3,Scott Vieira Jr,I,False,0.6,False,Tennessee,07,655.0,1049.0


In [65]:

url_list = []
for i in range(1,10):
    url_list.append(f"https://www.opensecrets.org/races/candidates?cycle=2020&id=TN0{i}&spec=N")
url_list

['https://www.opensecrets.org/races/candidates?cycle=2020&id=TN01&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN02&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN03&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN04&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN05&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN06&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN08&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN09&spec=N']

#### Looping through districts 

In [66]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
affiliation_list = []
incumbent_list = []
percentage_list = []
winner_list = []
candidate_list = []
state_list = []
district_list = []
Money_raised = []
Money_spent = []

for url in url_list:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    Members = soup.findAll(class_ = 'Members--bio u-richtext')
    state_district = soup.findAll(class_ = 'Hero-title')
    Money = soup.findAll(class_ = 'Members--number')
    
    incumbent_list.extend(get_incumbent(Members))
    percentage_list.extend(get_percentage(Members))
    winner_list.extend(get_winner(Members))
    candidate_list.extend(get_candidate_info(Members)[0])
    affiliation_list.extend(get_candidate_info(Members)[1])
    state_list.extend(get_state_district(state_district)[0])
    district_list.extend(get_state_district(state_district)[1])
    Money_raised.extend(get_money_details(Money)[0])
    Money_spent.extend(get_money_details(Money)[1])


TN_df = pd.DataFrame({"Candidate Name" : candidate_list,
                             "Affiliation" : affiliation_list,
                             "Incumbent": incumbent_list,
                             "Percentage of Vote" : percentage_list,
                             "Winner" : winner_list,
                             "State": state_list,
                             "District" : district_list,
                             "Money Raised": Money_raised,
                             "Money Spent": Money_spent}
                             )


In [67]:
TN_df.head()

,Candidate Name,Affiliation,Incumbent,Percentage of Vote,Winner,State,District,Money Raised,Money Spent
0,,,False,74.8,True,Tennessee,01,2126946.0,1869100.0
1,Blair Nicole Walsingham,D,False,22.4,False,Tennessee,01,140209.0,134995.0
2,Tim Burchett,R,True,67.7,True,Tennessee,02,1336276.0,878488.0
3,Renee Hoyos,D,False,31.0,False,Tennessee,02,812784.0,816793.0
4,Chuck Fleischmann,R,True,67.3,True,Tennessee,03,1051653.0,381411.0


In [68]:
TN_df["Affiliation"].unique()

array(['', 'D', 'R', 'I'], dtype=object)

### Get the list of Districts of each state

In [69]:

url_ = "https://en.wikipedia.org/wiki/2020_United_States_House_of_Representatives_elections"
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
page_ = requests.get(url_)
soup_ = BeautifulSoup(page_.content, "html.parser")

Districts = soup_.findAll('table',attrs = {'class': 'wikitable', 'class': 'sortable'})
districts_rows = Districts[0].findAll("tr")
districts_rows = districts_rows[2:52]

states_list = []
no_districts = []
for th in districts_rows:
    states_list.append(th.find("th").text.strip())
    no_districts.append(th.find("td").text.strip())

states_df = pd.DataFrame({"State" : states_list,
                             "District" : no_districts}
                             )


In [70]:
states_df.head()

,State,District
0,Alabama,7
1,Alaska,1
2,Arizona,9
3,Arkansas,4
4,California,53


In [71]:
states_abbv = pd.read_csv("../Data/states.csv", delimiter= ",").rename(columns={"Abbreviation" : "State_abbv"})

In [72]:
states_district = states_abbv.merge(states_df, on = "State", how = "inner")
states_district.head()

,State,State_abbv,District
0,Alabama,AL,7
1,Alaska,AK,1
2,Arizona,AZ,9
3,Arkansas,AR,4
4,California,CA,53


### Expand it to all the states and districts

In [73]:
full_url_list = []
for _, row in states_district.iterrows():
    for i in range(1, int(row['District'])+1):
        full_url_list.append(f"https://www.opensecrets.org/races/candidates?cycle=2020&id={row['State_abbv']}{i:02d}&spec=N")


In [74]:
len(full_url_list)

435

In [75]:
full_url_list

['https://www.opensecrets.org/races/candidates?cycle=2020&id=AL01&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL02&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL03&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL04&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL05&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL06&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL07&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AK01&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ01&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ02&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ03&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ04&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AZ05&spec=N',
 'https://www.opensecrets

In [76]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
affiliation_list = []
incumbent_list = []
percentage_list = []
winner_list = []
candidate_list = []
state_list = []
district_list = []
Money_raised = []
Money_spent = []

for url in full_url_list:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    Members = soup.findAll(class_ = 'Members--bio u-richtext')
    state_district = soup.findAll(class_ = 'Hero-title')
    Money = soup.findAll(class_ = 'Members--number')
    
    incumbent_list.extend(get_incumbent(Members))
    
    percentage_list.extend(get_percentage(Members))
    winner_list.extend(get_winner(Members))
    candidate_list.extend(get_candidate_info(Members)[0])
    affiliation_list.extend(get_candidate_info(Members)[1])
    state_list.extend(get_state_district(state_district)[0])
    district_list.extend(get_state_district(state_district)[1])
    Money_raised.extend(get_money_details(Money)[0])
    Money_spent.extend(get_money_details(Money)[1])


All_data = pd.DataFrame({"Candidate Name" : candidate_list,
                             "Affiliation" : affiliation_list,
                             "Incumbent": incumbent_list,
                             "Percentage of Vote" : percentage_list,
                             "Winner" : winner_list,
                             "State": state_list,
                             "District" : district_list,
                             "Money Raised": Money_raised,
                             "Money Spent": Money_spent}
                             )


TypeError: 'float' object is not callable

In [ ]:
url

In [ ]:
page.status_code

In [ ]:
soup

In [ ]:
Members

In [ ]:
Members[0]

def get_incumbent(Members):    
    incumbent_list = []
    for th in Members:
        if th[0].find('i').text == 'Incumbent':
            incumbent_list.append(1)
        else:
            incumbent_list.append(0)


In [ ]:
Members[0].find("i").text

In [ ]:
page = requests.get(url_0)
soup = BeautifulSoup(page.content, "html.parser")

Members = soup.findAll(class_ = 'Members--bio u-richtext')
# state_district = soup.findAll(class_ = 'Hero-title')
# Money = soup.findAll(class_ = 'Members--number')
    
# incumbent_list.extend(get_incumbent(Members))
    
# percentage_list.extend(get_percentage(Members))
# winner_list.extend(get_winner(Members))
# candidate_list.extend(get_candidate_info(Members)[0])
# affiliation_list.extend(get_candidate_info(Members)[1])
# state_list.extend(get_state_district(state_district)[0])
# district_list.extend(get_state_district(state_district)[1])
# Money_raised.extend(get_money_details(Money)[0])
# Money_spent.extend(get_money_details(Money)[1])

In [ ]:
Members[0]#.find("a").text.split()[-1]

In [ ]:
def get_incumbent(Members):    
    list = []
    for th in range(0,len(Members)):
     #print(th.text.strip())
        if Members[th].find("a").text.split()[-1] == "Incumbent":
            list.append(1)
        else:
            list.append(0)
    return list

In [ ]:
get_incumbent(Members)